*GSK NLP Analyst Technical Case Study
**Christine C. Amuzie
***Completed: 2021.12.10
***Last Modified: 2021.12.10

Select a drug from the dataset. Imagine that a leader for that product has asked you the following question: “What aspects do patients like about using our product, and what do people dislike?”
Use the data to write a report for this leader that will address their question.

In [ ]:
# Loading relevant libraries
import transformers
from transformers import BertModel, BertTokenizer, AdamW, get_linear_schedule_with_warmup
import torch
import numpy as np
import pandas as pd
import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from collections import defaultdict
from textwrap import wrap

In [7]:
# Data Exploration
%reload_ext watermark
%watermark -v -p numpy,pandas,torch,transformers

ModuleNotFoundError: No module named 'watermark'